Dieses jupyter Notebook dient dem analysieren und visualisieren der verwendeten Semantik von Bundestagsparteien im Plenarsaal.
Seine Funktion ist es alle gesprochenen Wörter herauszufiltern, die ausschließlich von einer Partei und nicht den anderen Parteien verwendet wurden, außerdem noch die von zwei Parteien geminsam und wiederum nicht von den anderen Parteien verwendet wurden. Das Ziel ist es, Parteien aufgrund ihres Jargons und semantischer Charakteristika zu unterscheiden, Zusammenhänge zu erkennen und eventuell einordnen zu können.
Als Datensatz dienen die digitalisierten Reden in den Plenarprotokollen des deutschen Bundestages der 19. Wahlperiode. Mit den analysierten Materialien lassen sich am Ende Wortgraphen erzeugen. Betrachtet werden die 6 Parteien, die über die 5 Prozent Klausel in den deutschen Bundestag gewählt wurden und ihn maßgeblich gestalten. Außen vor bleiben fraktionslose Kandidaten oder gesonderte Abgeordnete aus Bremen im Datensatz.

## Module und Bibliotheken einbinden

Ausgewählte Module und Bibliotheken werden bereitgestellt, die man später in den geschriebenen Methoden aufrufen kann.
Zuerst wird das json-Modul importiert um json-Dateien (JavaScript Object Notations; z.B. die Bundestagsredenliste) nutzen zu können.
Die Matplotlib ist ein Visualisierungsmodul. Zu ihr gehört auch das pyplot-Modul, welches gerangezogen wird um Darstellung ähnlich der bekannten Matlab-Platform zu ermöglichen. Als Pseudonym wird dazu fortlaufend alternativ plt verwenden.
Die Wordcloud bildet die Häufigkeit vom Vorkommen von Wörtern mittels einer Grafik ab.
PIL (Python Imaging Library) ist eine Bibliothek zum arbeiten mit verschiedenen Bilddateiformaten.
Um später die Grafik zu erstellen fügen wir ebenfalls die NumPy-Bibliothek hinzu.
Hinzukommt Pythons Objektzähler, der Counter, zum zählen der Wörter.
Als letzer Punkt wird, um die (sozialen) Netzwerke als Graph abbilden zu können, die Python-Bibliothek Networkx importiert.

In [ ]:
import json
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from PIL import Image
import numpy as np
from collections import Counter
import networkx as nx

## importieren der Reden und anwenden der Lemmata

Alle Reden der Sitzungen vom deutschen Bundestag der 19. Wahlperiode sind in Plenarprotokollen auf der Website des deutschen Bundestages in XML und json Dateien beziehbar. Zum schnelleren Bearbeiten dieser großen Menge an Daten (>24000 Reden), bietet sich für das das jupyter notebook das weit verbreitete und leicht verarbeitbare json-Format an.
Die Funktion laden_alleReden soll die vorbereitete json Datei mit dem Inhalt aller Reden laden und gibt eine Kopie der json Datei wieder damit...
Die Funktion laden_alleReden öffnet und ließt durch Aufruf des Kürzels fp das angegebene Verzeichnis. Dadurch wird der Inalt des Verzeichnisses geladen, in data gespeichert und eine Kopie für das Notebook erstellt.

In [ ]:
def laden_alleReden():
    
    with open('../data/speeches_preprocessed.json', 'r') as fp:
        data = json.load(fp)
    
    return data.copy()

Die Funktion laden_alleReden wird nun mit alleReden aufgerufen und packt den Return in die Liste??? alleReden.

In [ ]:
alleReden = laden_alleReden()

## Reden nach Parteien sortieren und zu Parteitexten zusammenfügen

Jeder Parteiname wird als ein Key deklariert und erhält ein Array, in dem man die Reden die zu dieser Partei gehören ablegen kann. Mittels der Attribute Redner und parteizugehörigkeit, die die gebündelten Reden in der json-Datei besitzen, werden alle Reden auf die jeweils zugehörige Partei aufgeteilt und dort zu einer Parteiliste ergänzt, bis alle Reden verteilt wurden.
Zur erheblichen Reduzierung der Rechenzeiten, wurden auf dem Text zuvor Lemmata angewendet???, um Mehrfachauftreten von Wörtern durch angrenzende Satzzeichen zu verhindern.
Protokollseitig wurden die Parteinamen nicht immer identisch notiert, weshalb abweichende Namensvariationen zusätzlich den üblichen Parteibezeichnungen zugewiesen werden müssen.

In [ ]:
reden_gefiltert = {'CDU_CSU':[],
                   'SPD':[],
                   'AfD':[],
                   'FDP':[],
                   'BÜNDNIS 90_DIE GRÜNEN':[],
                   'DIE LINKE':[],
                   'fraktionslos':[],
                   'Bremen':[]
                  }

for rede in alleReden:
    rede['party']=rede['party'].replace(u'\xa0', u' ')
    rede['party']=rede['party'].replace(u'/', u'_')
    if rede['party']=='Bündnis 90_Die Grünen':
        rede['party']='BÜNDNIS 90_DIE GRÜNEN'
    if rede['party']=='Fraktionslos':
        rede['party']='fraktionslos'
    
    reden_gefiltert[ rede['party'] ].extend(rede['text_lem'])

## erstellen von unique words (uw) von Einzelparteien und Parteipaaren

Mit dieser Funktion sollen jene Wörter gefunden werden, die ausschließlich von einer Partei stammen und somit ihr Merkmal sind. Diese zuweisbaren Wörter werden im Folgenden unique words genannt. Das Herausfiltern dieser "charakteristischen" Wörter geschieht mit Hilfe einer einfachen Mengensubtraktion.
Die Funktion verwendet die Parteinamen in Form der erstellten Keys und bildet zwei Wörterbücher in Form von Sets.
Das eine enthält Reden der zu untersuchenden Partei(Key) und das andere die wieder gebündelten Reden der restlichen Parteien.
Die unique words der zu untersuchenden Partei sind das Resultat der Subtraktion der Wörter aus der Menge der eigenen Parteireden minus der Wörter aus der Menge der Parteireden der restlichen Parteien.
Am Ende werden die unique Words in Form von Listen ausgegeben.

In [ ]:
def return_unique_words(partylist,word_dict):

    andereworte = set()
    parteienworte = set()
    for key in word_dict.keys():
        if key not in partylist:
            andereworte = andereworte | set(word_dict[key])
        else:
            parteienworte = parteienworte | set(word_dict[key])
    
    

    unique_words = parteienworte - andereworte
    
    return unique_words

## unique words nach Parteien in Form von Keys sortiert

In dem Set uw_all werden nun die unique words aller Parteien heruasgefiltert und unter ihren Partei Keys abgelegt, wie eine sortierte Tabelle.

In [ ]:
uw_all = {'SPD': return_unique_words('SPD',reden_gefiltert),
'FDP': return_unique_words('FDP',reden_gefiltert),
'CDU_CSU': return_unique_words('CDU_CSU',reden_gefiltert),
'DIE LINKE': return_unique_words('DIE LINKE',reden_gefiltert),
'BÜNDNIS 90_DIE GRÜNEN': return_unique_words('BÜNDNIS 90_DIE GRÜNEN',reden_gefiltert),
'AfD': return_unique_words('AfD',reden_gefiltert)
}

Die gesammten unique Words einer Partei lassen sich jetzt mit Angabe des Partei Keys öffnen.

In [ ]:
print(uw_all['SPD'])

## speichern

Die Liste der unique words einer jeden Partei wird als json Datei im Verzeichnis bereitgestellt.
Zuerst wird eine Leere Menge erschaffen, die Listen werden zurückgegeben, mit dem Inhalt, der bei uw_all unter dem Parteinamen als Key gespeichert ist.
Mittels der open Funktion wird ermöglicht bei eingabe des Kürzels unter eingetragenem Dateipfad eine json Datei zu schreiben. In dieser json Datei wird dann alles aus der Liste aus dem oberen Teil abgespeichert und im Verzeichnis abgelegt.

In [ ]:
uw_all_lists = {}
for key in uw_all:
    uw_all_lists.update({ key : list(uw_all[key]) })

with open('../data/unique_words_per_party.json', 'w') as fp:
    json.dump(uw_all_lists, fp, sort_keys=True, indent=4)

## erstellen von shared unique words (suw) von Parteipaaren

Um weitere Charakteristika in der Semantik parteiübergeordneter politischer Blöcke zu untersuchen, werden die gemeinsamen unique words paarweiser Parteien erstellt. Also erst eine Schnittmenge der Wörter die zwei Parteien vereinen und dann mit einem gesammelten Wörterbuch der übrigen Parteien vergleichen. Somit lässt sich eventuell eine sprachliche Nähe un dgemeinsamer häufiger Gebrauch bestimmter Wörter zwischen Parteien ausfindig machen.
Die unique words listen werden wie ein Schachbrett in Reihe und Spalte aufgelistet und die Zweierkombinationen ertsellt während die unique words Listen der übrigen 4 Parteien wieder gebündelt subtrahiert werden.
Dazu wird in aufsteigender Zählweise mit der Partei in der i. Spalte begonnen und für die erstmal alle Kombinationsmöglichkeiten mit den Parteien in den j Reihen erstellt, mit j einer Spaltenzahl höher beginnend, damit keien Schnittmenge mit sich selbst entsteht, bzw. die Diagonal der Tabelle ausgelassen wird. Da die Parteikeys aus der uw_all Liste keinen Zahlen in Rangfolgen entsprechen, wird mit enumerate nach Anzahl der Keys, einem jeden Partei Key eine Zahl zugewiesen unter der er sich dann in Spalte und Reihe wiederfindet.
Der print Befehl gibt zudem die Anzahl der shared unique words einer jeden Partei mit den jeweils anderen Parteien wieder.

In [ ]:
rows, cols = (len(uw_all), len(uw_all))
suw = [[0 for i in range(cols)] for j in range(rows)]
suw_words = [[0 for i in range(cols)] for j in range(rows)]
for ix,p1 in enumerate(uw_all.keys()):
    for jx,p2 in enumerate(uw_all.keys()):
        if jx > ix:
            print(p1,p2)
            tmp = return_unique_words([p1,p2],reden_gefiltert) - return_unique_words(p1,reden_gefiltert) - return_unique_words(p2,reden_gefiltert)
            suw_words[ix][jx] = tmp
            suw[ix][jx] = len(tmp)
            suw[jx][ix] = suw[ix][jx]

print(suw)

#0 SPD, 1 FDP, 2 CDU/CSU, 3 Linke, 4 Grüne, 5 AfD

Die unique words eines bestimmten Parteienpaares lassen sich jetzt mit Angabe der Parteitabellenzahlen öffnen.

In [ ]:
#print(uw_all.keys())
#print(suw_words[0][1])
#0 SPD, 1 FDP, 2 CDU/CSU, 3 Linke, 4 Grüne, 5 AfD

## Wordcloud einer einzelnen Partei

Nun wird aus den Listen zur Veranschaulichung eine Wordcloud erstellt. Bei einer Wordcloud entspricht die Schriftgröße der dargestellten Wörter ihrer prozentualen Häufigkeit des Vorkommens in der Liste.
Es wird der Partei-Key der zu visualisierenden Partei verwendet.
in der Counter Funktion werden die Wörter der unique word Liste der Partei iterativ durhgegangen und bei jedem geschaut, wie oft es im ungekürzten Parteitext vorkommt, der all ihre Reden enthällt um die Häufigkeit des verwendeten Wortes festzustellen.
Die imagemask bzw. Formstruktur der Wordcloud wird durch ein selbstgewähltes Bild (Kreis) festgelegt, wobei ein png Dateiformat notwendig ist.
Weiter lassen sich als Grundeinstellung ein paar Parameter festlegen, wie Detailgrad und Farben und die Quelle für die anzuzeigenden Wörter in Form der Counterausgabe.
Aus den Grundeinstellungen wird jetzt die Cloud, bzw. in diesem Fall der Kreis erzeugt.

In [ ]:
#party = 'AfD'
#party = 'CDU_CSU'
#party = 'SPD'
#party = 'FDP'
#party = 'BÜNDNIS 90_DIE GRÜNEN'
party = 'DIE LINKE'

uw_counter = [wort for wort in reden_gefiltert[party] if wort in uw_all[party]]
counts = Counter(uw_counter)

mask = np.array(Image.open("circle.png"))

# parameters:

wordcloud = WordCloud(width=3200, height=3200,background_color='white', max_words=500, mask=mask,contour_color='#000000',contour_width=0,colormap='rainbow').generate_from_frequencies(counts)


# create image as circle:

fig = plt.figure(figsize=(10,10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")

# presentate:

plt.show()

## speichern als Rasterbild

Genügt die Darstellung den Ansprüchen, ist sie als "normales Bild", als Rasterbild im png Format auf dem Dateipfad mit dem jeweiligen Dateinamen am Ende abspeicherbar.

In [ ]:
# store to file
wordcloud.to_file(f"../private/wordclouds/wordcloud_{party}.png")
#plt.savefig(f"../private/wordclouds/wordcloud_{party}.png", format="png")

## speichern als SVG

Für die Zwecke einer qualitätsverlustfreien Vergrößerung der Wordcloud zum pixelfreien Heranzoomen, oder
z.B. Anbringung auf Postern, lässt sich die Darstellung auch als Vektorgrafik in Form einer SVG Datei abspeichern. 
w+ = Datei lesen und schreiben.

In [ ]:
party='DIE GRÜNEN'
wordcloud_svg = wordcloud.to_svg(embed_font=True)
f = open(f"../private/wordclouds/wordcloud_{party}.svg","w+")
f.write(wordcloud_svg)
f.close()

## Wordcounterliste und Cloud für Parteipaar

Die Funktion erstellt eine Wordcloud aller selbst gewählten Parteipaare.

In [ ]:
#0 SPD, 1 FDP, 2 CDU/CSU, 3 Linke, 4 Grüne, 5 AfD

uw_counter = [wort for wort in reden_gefiltert['SPD'] + reden_gefiltert['FDP'] if wort in suw_words[0][1]]

counts = Counter(uw_counter)

mask = np.array(Image.open("circle.png"))

# parameters:

wordcloud = WordCloud(background_color='white', max_words=500, mask=mask,contour_color='#000000',contour_width=3,colormap='rainbow').generate_from_frequencies(counts)
#wordcloud = WordCloud(background_color='white', max_words=100, mask=mask).generate_from_frequencies(counts)


# create image as circle

fig = plt.figure(figsize=(10,10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")

# presentate:

plt.show()

## speichern als SVG

Funktion zum Abspeichern der Wordcloud der Parteipaare als Vektorgrafik.

In [ ]:
wordcloud_svg = wordcloud.to_svg(embed_font=True)
f = open("../private/wordclouds/wordcloud.svg","w+")
f.write(wordcloud_svg )
f.close()

## Graph erstellen

Als besondere Darstellungsmethode von (sozilen) Netzwerken als Graph, kann ein Netzwerk ausgegeben werden, das sich mit der Software Gephi visualisieren lässt. Dafür wird im jupyter notebook ein Knoten- und Kantennetzwerk aus den unique words Listen erzeugt. Als erstes werden die Knoten erstellt und mit den leeren Attributen Identifikationsnummer, wird erzeugt durch enumerate-Schleife, dazu Parteinamen und der Größe der parteiangehörigen unique words Liste ausgestattet.
Append fügt der Liste "nodes" jeden Knoten als Item hinzu. Knoten werden befüllt.

In [ ]:
nodes = []
for px,partei in enumerate(uw_all.keys()):
    node = {
        'id' : px,
        'name' : partei,
        'nuw' : len(uw_all[partei])
    }
    nodes.append(node)
    
nodes

Ein Graph wird erstellt, der ist nicht gerichtet, wird word_graph deklariert, bekommt die vorher erstellten Knoten als nodes und noch zu berechnende Links als Kanten des Graphen.
Für die Links bzw. KAnten, werden alle Knoten miteinander verglichen. Es gibt über jede Kante 2 inzidente Knoten, einer als Quelle einer als Ziel, dazu werden Gewichte berechnet, auf BAsis der geteilten unique words (suw).
Je mehr geteilte Wörter es gibt, desto stärker ist das Gewicht zwischen den Parteien.
Jede berehnete KAnte wird dem Graphen hinzugefügt.

In [ ]:

graph = {
    'directed': False,
    'graph': 'word_graph',
    'links': [],
    'nodes': nodes
}    

for ix,nodeI in enumerate(graph['nodes']):
    for jx,nodeJ in enumerate(graph['nodes']):
        if jx > ix:
            #print(p1,p2)
            #tmp = return_unique_words([p1,p2],reden_gefiltert) - return_unique_words(p1,reden_gefiltert) - return_unique_words(p2,reden_gefiltert)
            #suw_words[ix][jx] = tmp
            weight = suw[ix][jx]
            #suw[jx][ix] = suw[ix][jx]
            
            source = nodeI['id']
            target = nodeJ['id']
            link_dict = {
                    'source':source,
                    'target':target,
                    'weight':weight       
                }
            graph['links'].append(link_dict)
    
print(graph)

Jeder Knoten wird an Gephi übergeben, bis alle Knoten fertig sind. Dann folgen die Kanten.
Eine XML wird im selben Ordner wie das Notebook abgespeichert.

In [ ]:
graphforgephi = nx.Graph()
for node in graph['nodes']:
    graphforgephi.add_node(node['id'],name = node['name'],nuw = node['nuw']);
    #if(node['name'] == 'Angela Merkel'):
    #    print(node['tops'])

print('nodes done')
for link in (graph['links']):   
    graphforgephi.add_edge(link['source'],link['target'],weight = link['weight'])
print('links done')   
nx.write_gexf(graphforgephi, "graphforgephi.gexf")
print('save done')